<h1>Webscraping Bitcoin</h1>

In [1]:
# Setup configs
import yfinance as yf
import schedule
import time
import datetime
import boto3
import json
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# Configurar o cliente Kinesis Firehose
firehose_client = boto3.client(
    'firehose', 
    region_name='us-east-1') # Substitua pela região onde o Firehose foi criado
delivery_stream_name = 'your_delivery_stream_name' # Substitua pelo nome do seu Delivery Stream


def fetch_and_send_bitcoin_data():
    """
    Obtém dados de Bitcoin usando yfinance e envia para o Kinesis Firehose.
    """
    # Obter dados do Bitcoin usando yfinance
    bitcoin = yf.Ticker("BTC-USD")
    data = bitcoin.history(period="1m")
    
    # Pegar o último registro (mais recente)
    latest_data = data.iloc[-1]
    timestamp = datetime.datetime.now().isoformat()
    close_price = latest_data['Close']

    # Criar o registro JSON
    record = {
        'timestamp': timestamp,
        'close_price': close_price
    }

    # Enviar o registro para o Kinesis Firehose
    response = firehose_client.put_record(
        DeliveryStreamName=delivery_stream_name,
        Record={
            'Data': json.dumps(record) + '\n'
        }
    )
    
    print(f"Timestamp: {timestamp}, Close: {close_price}, Response: {response}")

# Agendar a tarefa para rodar a cada minuto
schedule.every(1).minute.do(fetch_and_send_bitcoin_data)

print("Iniciando a captura de dados do Bitcoin a cada minuto...")

# Loop para manter o script rodando
while True:
    schedule.run_pending()
    time.sleep(1)